# Exercise 2.2: A simple CNN for the edge lover task
<img src="https://raw.githubusercontent.com/tensorchiefs/dl_course_2024/refs/heads/main/notebooks/art_lover.png" alt="Sample Image" width="150">

**Storytime:** 
Es gibt einen Kunstliebhaber, der nur Bilder mit vertikalen Streifen mag. Das Ziel dieses Notizbuchs ist es, einen Algorithmus (CNN) zu verwenden, der dabei hilft, zu klassifizieren, ob der Kunstliebhaber das Bild mag oder nicht.

**Aufgabe:**
Sie trainieren ein sehr einfaches CNN mit nur einem Kernel, um zwischen Bildern mit vertikalen und Bildern mit horizontalen Streifen zu unterscheiden. Um zu überprüfen, welches Muster vom trainierten Kernel erkannt wird, visualisieren Sie die Gewichte des Kernels als Bild. Sie werden sehen, dass das CNN einen nützlichen Kernel lernt (entweder einen vertikalen oder einen horizontalen Balken). Sie können mit dem Code experimentieren, um den Einfluss der Kernelgröße, der Aktivierungsfunktion und der Pooling-Methode auf das Ergebnis zu überprüfen.

**Datensatz:** Sie arbeiten mit einem künstlich generierten Datensatz aus Graustufenbildern (50 x 50 Pixel) mit 10 vertikalen oder horizontalen Streifen. Wir möchten diese Bilder danach klassifizieren, ob der Kunstliebhaber, der nur vertikale Streifen mag, das Bild mögen wird (y = 0) oder nicht mögen wird (y = 1).  

Die Idee des Notebooks ist, dass Sie versuchen, den bereitgestellten Code zu verstehen, indem Sie ihn ausführen, die Ausgabe überprüfen und damit experimentieren, indem Sie den Code leicht ändern und erneut ausführen.  

___

### 2.2.1 Datensatz generieren

Schreiben Sie eine Funktion, die einen künstlichen Datensatz aus Schwarz-Weiß-Bildern (50 x 50 Pixel) mit 10 vertikalen oder horizontalen Balken (10 Pixel lang) erstellt. Verwenden Sie diese Funktion, um einen Trainingsdatensatz mit 1000 Beispielen zu erstellen, davon 500 vertikale und 500 horizontale Beispiele. Erstellen Sie auf ähnliche Weise einen Validierungsdatensatz mit 1000 Beispielen. Normalisieren Sie die Pixelwerte so, dass sie zwischen 0 und 1 liegen.
Sie sollten die folgende Ausgabe erhalten:

```{python}
print(X_train.shape) #(1000, 50, 50, 1)
print(np.unique(Y_train)) #[0 1]
```

In [ ]:
def create_img(n:int=1000, n_beam:int=10, x_dim:int=50, y_dim:int=50, line_len:int=10, line_space:bool=True, seed:int=None) -> tuple:
    """
    Create a dataset of images with vertical and horizontal lines.
    
    Parameters
    -----------
    n : int
        Number of images to create (should be even)
    n_beam : int
        Number of lines (beams) per image
    x_dim : int
        Width of the images
    y_dim : int
        Height of the images
    line_len : int
        Length of each line
    line_space : bool
        If True, ensures that lines are not adjacent to each other.
    seed : int or None
        Optional random seed for reproducibility. If provided, NumPy's RNG is seeded.
    
    Returns
    -----------
    X : np.ndarray
        Array of shape (n, x_dim, y_dim, 1) containing the images
    Y : np.ndarray
        Array of shape (n, 1) containing the labels (1 for vertical lines, 0 for horizontal lines)

    Raises
    -----------
    ValueError
        If n_beam is too large for the given image dimensions and line length.
    """
    
    import numpy as np

    # create a local random Generator for reproducibility when provided
    # using a Generator avoids altering global RNG state
    if seed is not None:
        rng = np.random.default_rng(seed)
    else:
        rng = np.random.default_rng()

    # check if n_beam is not too large
    if line_space:
        if n_beam * 3 > min(x_dim, y_dim):
            raise ValueError("n_beam is too large for the given image dimensions and line length.")
    else:
        if n_beam > min(x_dim, y_dim):
            raise ValueError("n_beam is too large for the given image dimensions.")

    # initialize arrays (images and labels)
    X = np.zeros((n, x_dim, y_dim, 1))
    Y = np.zeros((n, 1))

    # create n/2 images with n_beam vertical lines
    for i in range((n//2)):
        x_start_list = []
        for _ in range(n_beam):
            # check if x_start is not too close to existing lines
            x_start = int(rng.integers(0, x_dim))
            while (x_start) in x_start_list or (((x_start + 1) in x_start_list or (x_start - 1) in x_start_list) and line_space):
                x_start = int(rng.integers(0, x_dim))
            x_start_list.append(x_start)
            
            y_start = int(rng.integers(0, y_dim - line_len))
            for k in range(line_len):
                X[i, x_start, y_start + k, 0] = 1.0
        Y[i] = 1.0

    # create n/2 images with n_beam horizontal lines
    for i in range((n//2), n):
        y_start_list = []
        for _ in range(n_beam):
            # check if y_start is not too close to existing lines
            y_start = int(rng.integers(0, y_dim))
            while y_start in y_start_list or (((y_start + 1) in y_start_list or (y_start - 1) in y_start_list) and line_space):
                y_start = int(rng.integers(0, y_dim))
            y_start_list.append(y_start)

            x_start = int(rng.integers(0, x_dim - line_len))
            for k in range(line_len):
                X[i, x_start + k, y_start, 0] = 1.0
        Y[i] = 0.0

    return X, Y

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# define seed, set to None if not used
SEED = 42

# create dataset as defined above
X_train, Y_train = create_img(1000, seed=SEED)
X_val, Y_val = create_img(1000, seed=SEED)
print(X_train.shape, Y_train.shape)
print(np.unique(Y_train))

# visualize some random samples from the dataset
id = np.random.choice(len(X_train), 10, replace=False)
# test seed function, always plot first 10 images
#id = np.arange(10)
fig, axes = plt.subplots(2, 5, figsize=(15, 8))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_train[id[i]].reshape(50, 50), cmap='gray')
    ax.set_title(f"Label: {Y_train[id[i]]}")
    ax.axis('off')
plt.tight_layout()
plt.show()

___

### 2.2.2 Erstellen Sie ein möglichst einfaches CNN.

Erstellen Sie ein CNN mit einer Faltung von 5x5 (*convolution*) und einer Ausgabe. Trainieren Sie das CNN mit den Daten aus [2.2.1](#221-datensatz-generieren). Sie sollten nicht mehr als 28 trainierbare Parameter (siehe `model.summary()`) im Netzwerk haben.

* Zeichnen Sie die Lernkurven auf: (Epochen vs. Trainingsverlust und Validierungsverlust) und Epochen vs. Genauigkeit. Sie sollten eine Genauigkeit von ungefähr 1 erhalten.

**Hinweis**: Verwenden Sie die Max-Pooling-Operation auf clevere Weise.

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

In [ ]:
import keras

# Inspect Keras/TensorFlow versions and backend
print("KERAS_BACKEND env:", os.environ.get("KERAS_BACKEND"))
print("keras.__version__:", getattr(keras, "__version__", "unknown"))
try:
    from keras.backend import backend as kb_backend
    print("keras.backend.backend():", kb_backend())
except Exception as e:
    print("keras.backend.backend() unavailable:", e)
try:
    import tensorflow as tf
    print("tensorflow.__version__:", tf.__version__)
except Exception as e:
    print("TensorFlow import failed:", e)

In [ ]:
from keras import layers
from keras.models import Sequential
from keras.optimizers import SGD

Im ersten Schritt wird das Modell erstellt und ein Input-Layer mit der Form (50, 50, 1) definiert (50x50 Pixel, 1 Kanal für Graustufenbilder).

In [ ]:
model = Sequential()

model.add(layers.InputLayer(shape=(50, 50, 1)))

Anschließend wird eine Faltungsschicht mit einem 5x5-Kernel und der ReLU-Aktivierungsfunktion hinzugefügt. Das rezeptive Feld dieses Kernels ist also 5x5 Pixel groß mit welchem das Bild "gescannt" wird. Da lediglich ein Filter genutzt wird berechnet sich die Anzahl der trainierbaren Parameter wie folgt:
$$
\text{Anzahl der Parameter} = (\text{Kernelhöhe} \times \text{Kernelbreite} \times \text{Eingangskanäle} + 1) \times \text{Anzahl der Filter}
$$
$$= (5 \times 5 \times 1 + 1) \times 1 = 26
$$

In [ ]:
model.add(layers.Conv2D(1, (5, 5), activation='relu'))

In [ ]:
model.add(layers.MaxPooling2D((46, 46)))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from keras import utils
Y_train_cat = utils.to_categorical(Y_train)
Y_valid_cat = utils.to_categorical(Y_val)

history = model.fit(x=X_train, y=Y_train, validation_data=(X_val, Y_val), epochs=10, batch_size=10)

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot accuracy
ax1.plot(history.history['accuracy'], label='Train Accuracy')
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_title('Model Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend(loc='lower right')

# Plot loss
ax2.plot(history.history['loss'], label='Train Loss')
ax2.plot(history.history['val_loss'], label='Validation Loss')
ax2.set_title('Model Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend(loc='upper right')

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Predict the labels for the validation set
Y_pred = model.predict(X_val)
Y_pred_classes = (Y_pred > 0.5).astype(int)

# Compute the confusion matrix
cm = confusion_matrix(Y_val, Y_pred_classes)


# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Horizontal', 'Vertical'], yticklabels=['Horizontal', 'Vertical'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

___

### 2.2.3 Visualisieren Sie den gelernten Kernel
Um den gelernten Kernel zu visualisieren, können Sie `model.get_weights()` verwenden. 

Ist der gelernte Kernel sinnvoll?

In [ ]:
weights = model.get_weights()
print(weights[0].shape)
print(weights[1].shape)
print(weights[2].shape)
print(weights[3].shape)

In [ ]:
conv_weights = weights[0].reshape(5, 5)

import matplotlib.pyplot as plt
plt.imshow(conv_weights, cmap='gray')
plt.colorbar()
plt.title('Learned Convolutional Filter Weights')
plt.show()